In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time

# text NLP
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
from wordcloud import WordCloud
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from unidecode import unidecode

# Preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# model
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

# Score
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold

# Cleaning & Preprocessing

In [3]:
# import nltk
# import subprocess

# # Download and unzip wordnet
# try:
#     nltk.data.find('wordnet.zip')
# except:
#     nltk.download('wordnet', download_dir='/kaggle/working/')
#     command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
#     subprocess.run(command.split())
#     nltk.data.path.append('/kaggle/working/')

# # Now you can import the NLTK resources as usual
# from nltk.corpus import wordnet

In [4]:
# class BasicTextCleaning:
#     def __init__(self):
#         # define some necessary elements
#         self.stopwords = set(stopwords.words('english'))
#         self.words_corpus = set(words.words())
#         self.stemmer = PorterStemmer()
#         self.lemmatizer = WordNetLemmatizer()

#         # dictionary of methods can be used
#         self.methods = {'lowercase': str.lower,
#                         'accent_removal': self.accent_removal,
#                         'strip': str.strip,
#                         'nice_display': self.nice_display,
#                         'tokenization': nltk.word_tokenize,
#                         'stemming': self.stemming,
#                         'lemmatization': self.lemmatization,
#                         'punctuation_removal': self.punctuation_removal,
#                         'stopwords_removal': self.stopwords_removal,
#                         'contractions_expand': self.contractions_expand,
#                         'nonsense_removal': self.nonsense_removal,
#                         'number_removal': self.number_removal}

#         self.punctuations = '[%s]' % re.escape(string.punctuation)

#     def text_cleaning(self, texts, methods=None):
#         if not methods:
#             methods = ['accent_removal', 'lowercase', 'nice_display', 'punctuation_removal',
#                        'stopwords_removal', 'lemmatization', 'stemming']
#         if isinstance(texts, str):
#             texts = [texts]
#         cleaned_texts = []
#         for text in texts:
#             for method in methods:
#                 if method not in self.methods.keys():
#                     raise Warning('Invalid method "{}". Basic text cleaning methods available: {}'.format(method, ", ".join(self.methods.keys())))
#                 text = self.methods[method](text)
#             cleaned_texts.append(text)
#         return cleaned_texts

#     def strip_text(self, text):
#         return text.strip()

#     def lowercase(self, text):
#         return text.lower()

#     def contractions_expand(self, text):
#         return contractions.fix(text)

#     def number_removal(self, text):
#         text = re.sub(r"[^a-zA-Z]", " ", text)
#         text = re.sub(r'\s+', ' ', text)
#         return text

#     def nice_display(self, text):
#         text = re.sub(r"([^\w\s([{\'])(\w)", r"\1 \2", text)
#         text = re.sub(r'\s+', ' ', text)
#         return text.strip()

#     def accent_removal(self, text):
#         text = unidecode(text)
#         return text

#     def punctuation_removal(self, text):
#         text = re.sub(self.punctuations, ' ', text)
#         text = re.sub(r'\s+', ' ', text)
#         return text.strip()

#     def stopwords_removal(self, text):
#         return " ".join([word for word in text.split() if word not in self.stopwords])

#     def stemming(self, text):
#         return " ".join([self.stemmer.stem(word) for word in text.split()])

#     def lemmatization(self, text):
#         return " ".join([self.lemmatizer.lemmatize(word) for word in text.split()])

#     def tokenization(self, text):
#         return nltk.word_tokenize(text)

#     def nonsense_removal(self, text):
#         return " ".join([word for word in text.split() if wordnet.synsets(word)])

In [6]:
# cleaning = BasicTextCleaning()
# data['text'] = cleaning.text_cleaning(data['text_'])
# data.head()

In [14]:
data = pd.read_csv('/kaggle/input/datacleaning/datacleaning.csv')


In [15]:
data['label'].replace('CG', 1, inplace=True)
data['label'].replace('OR', 0, inplace=True)

/tmp/ipykernel_34/384321868.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['label'].replace('CG', 1, inplace=True)
/tmp/ipykernel_34/384321868.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.

In [16]:
data = data.fillna('')

# Save result

In [6]:
# data_save = pd.DataFrame(columns=['data','length_used', 'feature_extraction', 'feature_selection', 'model', 'accuracy', 'f1', 'recall', 'precision', 'roc_auc', 'notes'])
# data_save.to_csv('results.csv', index=False)

In [26]:
def save_and_print(data, length_used, feature_extraction, feature_selection, model, scores):
    data_save = pd.read_csv('/kaggle/working/results.csv')
    for feature_extraction in feature_extractions:
        sc = str(feature_extraction)
        new_row = {'data': data, 'length_used': length_used, 
               'feature_extraction': feature_extraction, 
               'feature_selection': feature_selection, 
               'model': model, 'accuracy': np.mean(scores[sc]['accuracy']),
               'f1': np.mean(scores[sc]['f1']), 'recall': np.mean(scores[sc]['recall']), 
               'precision': np.mean(scores[sc]['precision']), 'roc_auc': np.mean(scores[sc]['roc_auc']), 
               'notes': notes}
        data_save.loc[len(data_save)] = new_row
        data_save.to_csv('/kaggle/working/results.csv', index=False)

# Case 1

In [17]:
dataname = 'Fake Review Dataset'
lengh_used = None
feature_selection = None
notes = None


In [18]:
X, y = data['text'], data['label']

# LGBM

In [38]:
import lightgbm as lgb

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [40]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(n_features = 10), HashingVectorizer(n_features = 50),
                     HashingVectorizer(n_features = 100),HashingVectorizer()]

folds = kf.split(X, y)
scores = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_lg = lgb.LGBMClassifier(force_row_wise=True)
        model_lg.fit(X_train_hashed, y_train)

        y_test_pred = model_lg.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_lg.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores:
            scores[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores[key]['accuracy'].append(accuracy)
        scores[key]['f1'].append(f1)
        scores[key]['recall'].append(recall)
        scores[key]['precision'].append(precision)
        scores[key]['roc_auc'].append(roc_auc)

[LightGBM] [Info] Number of positive: 16200, number of negative: 16145
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 32345, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500850 -> initscore=0.003401
[LightGBM] [Info] Start training from score 0.003401
[LightGBM] [Info] Number of positive: 16200, number of negative: 16145
[LightGBM] [Info] Total Bins 12750
[LightGBM] [Info] Number of data points in the train set: 32345, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500850 -> initscore=0.003401
[LightGBM] [Info] Start training from score 0.003401
[LightGBM] [Info] Number of positive: 16200, number of negative: 16145
[LightGBM] [Info] Total Bins 25499
[LightGBM] [Info] Number of data points in the train set: 32345, number of used features: 100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500850 -> initscore=0.003401
[LightGBM] [Info] Start training from score 0.003401
[LightG

In [53]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_lg, scores)

# XG boost

In [30]:
from xgboost import XGBClassifier

In [44]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(n_features = 10), HashingVectorizer(n_features = 50),
                     HashingVectorizer(n_features = 100),HashingVectorizer()]

folds = kf.split(X, y)
scores_xgb = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_xgb = XGBClassifier()
        model_xgb.fit(X_train_hashed, y_train)

        y_test_pred = model_xgb.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_xgb.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_xgb:
            scores_xgb[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_xgb[key]['accuracy'].append(accuracy)
        scores_xgb[key]['f1'].append(f1)
        scores_xgb[key]['recall'].append(recall)
        scores_xgb[key]['precision'].append(precision)
        scores_xgb[key]['roc_auc'].append(roc_auc)

In [56]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_xgb, scores_xgb)

# Adaboost

In [12]:
from sklearn.ensemble import AdaBoostClassifier

In [19]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(n_features = 10), HashingVectorizer(n_features = 50),
                     HashingVectorizer(n_features = 100),HashingVectorizer()]

folds = kf.split(X, y)
scores_ada = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_ada = AdaBoostClassifier(n_estimators=100, learning_rate=1.0)
        model_ada.fit(X_train_hashed, y_train)

        y_test_pred = model_ada.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_ada.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_ada:
            scores_ada[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_ada[key]['accuracy'].append(accuracy)
        scores_ada[key]['f1'].append(f1)
        scores_ada[key]['recall'].append(recall)
        scores_ada[key]['precision'].append(precision)
        scores_ada[key]['roc_auc'].append(roc_auc)

In [29]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_ada, scores_ada)

# CATBOOST

In [22]:
import catboost as cb

In [24]:
folds = kf.split(X, y)
scores_cat = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_cat = cb.CatBoostClassifier(
            random_state=42,
            task_type='GPU',
            border_count=2 * len(feature_extractions)
        )

        model_cat.fit(
            X_train_hashed,
            y_train,
            verbose=False,
            plot=False,
            early_stopping_rounds=50,
            use_best_model=True,
            eval_set=(X_test_hashed, y_test)
        )

        y_test_pred = model_cat.predict(X_test_hashed)

        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_cat.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_cat:
            scores_cat[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_cat[key]['accuracy'].append(accuracy)
        scores_cat[key]['f1'].append(f1)
        scores_cat[key]['recall'].append(recall)
        scores_cat[key]['precision'].append(precision)
        scores_cat[key]['roc_auc'].append(roc_auc)

In [27]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_cat, scores_cat)

# Logistic Regression

In [11]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(n_features = 10), HashingVectorizer(n_features = 50),
                     HashingVectorizer(n_features = 100),HashingVectorizer()]

folds = kf.split(X, y)
scores_lr = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_lr = LogisticRegression(max_iter = 1000)
        model_lr.fit(X_train_hashed, y_train)

        y_test_pred = model_lr.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_lr.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_lr:
            scores_lr[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_lr[key]['accuracy'].append(accuracy)
        scores_lr[key]['f1'].append(f1)
        scores_lr[key]['recall'].append(recall)
        scores_lr[key]['precision'].append(precision)
        scores_lr[key]['roc_auc'].append(roc_auc)

In [12]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model, scores)

# KNN

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(n_features = 10), HashingVectorizer(n_features = 50),
                     HashingVectorizer(n_features = 100),HashingVectorizer()]
metrics_param = ['cosine','minkowski']
neighbors_param = [5, 10, 50, 100]
folds = kf.split(X, y)
scores_knn = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)
        for metric in metrics_param:
            for neighbor in neighbors_param:
                model_knn = KNeighborsClassifier(metric = metric, n_neighbors = neighbor)
                model_knn.fit(X_train_hashed, y_train)

                y_test_pred = model_knn.predict(X_test_hashed)
                accuracy = accuracy_score(y_test, y_test_pred)
                f1 = f1_score(y_test, y_test_pred, pos_label=1)
                recall = recall_score(y_test, y_test_pred)
                precision = precision_score(y_test, y_test_pred)
                roc_auc = roc_auc_score(y_test, model_knn.predict_proba(X_test_hashed)[:, 1])

                key = str(feature_extraction)
                if key not in scores_knn:
                    scores_knn[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

                scores_knn[key]['accuracy'].append(accuracy)
                scores_knn[key]['f1'].append(f1)
                scores_knn[key]['recall'].append(recall)
                scores_knn[key]['precision'].append(precision)
                scores_knn[key]['roc_auc'].append(roc_auc)

In [ ]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_knn, scores_knn)

# Result

In [4]:
result = pd.read_csv('/kaggle/input/result-1/results_1.csv')
result = result.sort_values(by='accuracy', ascending=False)
result

,data,length_used,feature_extraction,feature_selection,model,accuracy,f1,recall,precision,roc_auc,notes
3,Fake Review Dataset,NaN,HashingVectorizer(),NaN,LogisticRegression(max_iter=1000),0.854472,0.854731,0.856315,0.853172,0.932805,NaN
30,Fake Review Dataset,NaN,HashingVectorizer(),NaN,<catboost.core.CatBoostClassifier object at 0x...,0.849030,0.848443,0.845242,0.851696,0.931995,NaN
34,Fake Review Dataset,NaN,HashingVectorizer(),NaN,<catboost.core.CatBoostClassifier object at 0x...,0.849030,0.848443,0.845242,0.851696,0.931995,NaN
22,Fake Review Dataset,NaN,HashingVectorizer(),NaN,LGBMClassifier(force_row_wise=True),0.844801,0.843937,0.839496,0.848431,0.930118,NaN
26,Fake Review Dataset,NaN,HashingVectorizer(),NaN,"XGBClassifier(base_score=None, booster=None, c...",0.839112,0.836559,0.823757,0.849778,0.926102,NaN
33,Fake Review Dataset,NaN,HashingVectorizer(n_features=100),NaN,<catboost.core.CatBoostClassifier object at 0x...,0.803868,0.808183,0.826445,0.790737,0.891575,NaN
29,Fake Review Dataset,NaN,HashingVectorizer(n_features=100),NaN,<catboost.core.CatBoostClassifier object at 0x...,0.803868,0.808183,0.826445,0.790737,0.891575,NaN
25,Fake Review Dataset,NaN,HashingVectorizer(n_features=100),NaN,"XGBClassifier(base_score=None, booster=None, c...",0.799391,0.803480,0.820276,0.787402,0.888400,NaN
21,Fake Review Dataset,NaN,HashingVectorizer(n_features=100),NaN,LGBMClassifier(force_row_wise=True),0.795978,0.796427,0.798369,0.794521,0.882648,NaN
38,Fake Review Dataset,NaN,HashingVectorizer(),NaN,AdaBoostClassifier(n_estimators=100),0.779358,0.779414,0.779717,0.779169,0.870845,NaN
